# Instruction

In this sample_code, we will explain the detail of using this package to generate your own chat dataset.

## Data Preparation

For this instruction, we prepared a sample dataset for you. Let's first take a look of this sample dataset, you can open the sample_dataset.xlsx through your xlsx reader(ex. Excel, GoogleSheet).  
You would see there are several sheet in this file: QuestionAskingMerge, Self_cognition, End_of_Conversation, PC and Screen
 
This file represents the example raw data format for users, you can follow the rules to prepare your own input data:  
1. Categorize your QA data into different sheets
2. Annotate the QA data with:
    - Type (category of question)
    - Level (level of question)
    - No (question number)
    - UID (Made by Level_No)
    - Parent (The parent QA UID)
    - Well-formed question and Well-formed answers  

    Type is the Category of this question  
    Level means the hierarchical status of this question, for example, the initial QA of a conversation should be A level (ex. Q: Hi, I wanna ask you a question. A: Hi, What you wanna ask?),  
    and the QA followed with A should be B level (ex. Q: Where is the capital of Taiwan? A: The capital of Taiwan is Taipei City), and so on.  
    In this beta version, we only provide four levels: A, B ,C and Z, where Z represents the final QA of this conversation(ex. Q: Thank you! A: You're welcome!)  
    You can follow the pattern of the sample file to get a more detail understand.

3. Put all the sheet(all different type of categories) into a merged sheet

## Generation

### Setup

After prepared your own dataset, you can follow the code below to generate your own chat dataset

First, let's import the packages we would use in this instruction, and set the path of files.

In [ ]:
# Packages
import pandas as pd
from chatgen.chat_algo import ChatAlgo
from chatgen.data_loader import load_xlsx, create_input_data

In [ ]:
# Set data path
input_file = "./dataset/sample_dataset.xlsx" # sample dataset or replace with your own dataset
sheet_name = 'QuestionAskingMerge' # the sheet we would use or replace with your own merged sheet name
output_file = "./output/conversations.json" # output path

And we can load the file by function ```create_input_data()``` (we provide ```load_csv()```for you to load the sample data, but you can also load your dataset by your own way into Pandas DataFrame)

In [ ]:
# Load raw data & Create input data
data = load_xlsx(input_file, sheet_name)
input_data = create_input_data(data)

In [ ]:
# Let's take a look of the input_data
input_data

### Algorithm

In this algorithm, we consider some factors to generate the chat dataset by simulate the behavior of human.  

First, you should consider the ```max_depth```, which is, how deep the conversation you wanna generate.   
For example, If you wanna finetune a chatbot that are an expert for debate, mostly debate is a long-term conversation, in this case, you should set the max_depth larger.  

Next, the ```init_weight``` is the probability of B, C levels, and ```final_level_weight``` is the Z level for initialization. Since mostly we start a conversation by A level, in default we set ```init_weight``` to be 0.05 and ```final_level_weight``` to be 10^-6, which generate good dataset based on our empiric.  

During the time of generation, the algorithm will follow the below pattern, repeat till generate total number of ```generate_times``` conversations:  

For each conversation in generated dataset:
1. Randomly choose a number from ```1```~```max_depth``` as the number of rounds of the conversations under generating
2. Initialize the sampling probability by ```init_weight```, ```final_level_weight``` and random choose the depth of this conversation from ```max_depth```
3. Sample one observation from input data base on the probability
4. Punish the probability of the observation we just sampled by ```current_punish```(since most of the time we don't repeat the same QA in real life)
5. If the observation have childs, reward its childs. Otherwise, reward the same level of the observation by ```child_reward``` (This will boost the conversation continue)
6. Modified the probability of levels by ```ORD``` matrix  
You can check ```./chatgen/chat_algo.py``` to see that matrix, by each row we get the weights of each level(column) base on observation's level(row), the value is base on our research empiric, this also can help boost the conversation continue
7. repeat 3~6. If the sample times reach ```final_weighting_threshold```, reward Z level by ```final_level_reward``` (people not always complete the conversation, there is a probability that conversation end before getting the answer they want)  

That's it! You can change the parameter base on your dataset to get the result you want.

In [ ]:
# Params setting
params = {
    'system_prompt':'You are a helpful artificial intelligent assistant, your name is ChatGenBot.',
    'generate_times': 1000,
    'max_depth': 6,
    'init_weight': 0.05,
    'final_level_weight':0.000001,
    'current_punish': 0.01,
    'child_reward': 8000,
    'final_weighting_threshold': 2,
    'final_level_reward': 5000,
}

By using ```create_chat_history()```, the it will generate chat history of input data with params you just set

In [ ]:
# Create conversation dataset
chat_algo = ChatAlgo(input_data) # initialization
chat_algo.create_chat_history(**params) # Generation

After generation, you can user ```sample_output()``` to take a glimpse of generated dataset

In [ ]:
chat_algo.sample_output()

Finally, you can save the result to json file with ```to_json()```

In [ ]:
chat_algo.to_json(output_file) # save to JSON